# BCSE209P - Machine Learning Lab

##**Divyansh**
##21BAI1240

## Lab 4 - Naive Bayes Classifier ( Applied On Wine quality Dataset )

###References
Website referred to make Naive Bayes Classifier From Scratch in Python - https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

Website referred to download the wine quality red Dataset - https://machinelearningmastery.com/standard-machine-learning-datasets/

In [9]:
from google.colab import files
uploaded= files.upload()

Saving winequality-red_final_1240.csv.csv to winequality-red_final_1240.csv.csv


In [11]:
import pandas as pd
import io
df= pd.read_csv(io.BytesIO(uploaded['winequality-red_final_1240.csv.csv']))
print(df)

       7.4    0.7     0  1.9  5
0      7.8  0.880  0.00  2.6  5
1      7.8  0.760  0.04  2.3  5
2     11.2  0.280  0.56  1.9  6
3      7.4  0.700  0.00  1.9  5
4      7.4  0.660  0.00  1.8  5
...    ...    ...   ...  ... ..
1593   6.2  0.600  0.08  2.0  5
1594   5.9  0.550  0.10  2.2  6
1595   6.3  0.510  0.13  2.3  6
1596   5.9  0.645  0.12  2.0  5
1597   6.0  0.310  0.47  3.6  6

[1598 rows x 5 columns]


### Read and Convert CSV Data to List of Floats


In [12]:
import csv

# Open the 'winequality-red' file in read mode using a context manager
with open('/content/winequality-red_final_1240.csv.csv', 'r') as read_obj:
    
    # Create a CSV reader object using the 'csv.reader()' function
    csv_reader = csv.reader(read_obj)

    # Convert the CSV data into a list of lists
    # Each row in the CSV file is converted into a list of float values
    list_of_dataset = [[float(item) for item in row] for row in csv_reader]

# Print the resulting list_of_dataset
print(list_of_dataset)

[[7.4, 0.7, 0.0, 1.9, 5.0], [7.8, 0.88, 0.0, 2.6, 5.0], [7.8, 0.76, 0.04, 2.3, 5.0], [11.2, 0.28, 0.56, 1.9, 6.0], [7.4, 0.7, 0.0, 1.9, 5.0], [7.4, 0.66, 0.0, 1.8, 5.0], [7.9, 0.6, 0.06, 1.6, 5.0], [7.3, 0.65, 0.0, 1.2, 7.0], [7.8, 0.58, 0.02, 2.0, 7.0], [7.5, 0.5, 0.36, 6.1, 5.0], [6.7, 0.58, 0.08, 1.8, 5.0], [7.5, 0.5, 0.36, 6.1, 5.0], [5.6, 0.615, 0.0, 1.6, 5.0], [7.8, 0.61, 0.29, 1.6, 5.0], [8.9, 0.62, 0.18, 3.8, 5.0], [8.9, 0.62, 0.19, 3.9, 5.0], [8.5, 0.28, 0.56, 1.8, 7.0], [8.1, 0.56, 0.28, 1.7, 5.0], [7.4, 0.59, 0.08, 4.4, 4.0], [7.9, 0.32, 0.51, 1.8, 6.0], [8.9, 0.22, 0.48, 1.8, 6.0], [7.6, 0.39, 0.31, 2.3, 5.0], [7.9, 0.43, 0.21, 1.6, 5.0], [8.5, 0.49, 0.11, 2.3, 5.0], [6.9, 0.4, 0.14, 2.4, 6.0], [6.3, 0.39, 0.16, 1.4, 5.0], [7.6, 0.41, 0.24, 1.8, 5.0], [7.9, 0.43, 0.21, 1.6, 5.0], [7.1, 0.71, 0.0, 1.9, 5.0], [7.8, 0.645, 0.0, 2.0, 6.0], [6.7, 0.675, 0.07, 2.4, 5.0], [6.9, 0.685, 0.0, 2.5, 6.0], [8.3, 0.655, 0.12, 2.3, 5.0], [6.9, 0.605, 0.12, 10.7, 6.0], [5.2, 0.32, 0.25, 1.

### Naive Bayes Classification with Cross-Validation


In [13]:
# Import necessary modules and functions
from csv import reader
from random import seed, randrange
from math import sqrt, exp, pi

# Load a CSV file and return the dataset as a list of lists
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert a specific column in the dataset from string to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert a specific column in the dataset from string to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split the dataset by class values and return a dictionary
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if class_value not in separated:
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated

# Calculate the mean of a list of numbers
def mean(numbers):
    return sum(numbers) / float(len(numbers))

# Calculate the standard deviation of a list of numbers
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x - avg) ** 2 for x in numbers]) / float(len(numbers) - 1)
    return sqrt(variance)

# Calculate the mean, standard deviation, and count for each column in the dataset
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del summaries[-1]
    return summaries

# Split dataset by class and calculate statistics for each row
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries

# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2] / float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

# Set the random seed for reproducibility
seed(1)

# Specify the filename of the CSV dataset
filename = '/content/winequality-red_final_1240.csv.csv'

# Load the CSV dataset
dataset = load_csv(filename)

# Convert the columns of the dataset from string to float
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)

# Convert the class column to integers
str_column_to_int(dataset, len(dataset[0]) - 1)

# Summarize the dataset by class
summaries = summarize_by_class(dataset)

# Calculate the probabilities of predicting each class for the first row of the dataset
probabilities = calculate_class_probabilities(summaries, dataset[0])

# Print the calculated probabilities
print(probabilities)

{0: 0.04125214602523575, 4: 0.01688482511873188, 3: 0.0005056142663373297, 2: 0.003948622887266587, 1: 6.372090887350758e-05, 5: 0.000392443722173896}


### Naive Bayes Classification with Accuracy Evaluation

In [17]:
# Import necessary modules and functions
from csv import reader
from random import seed, randrange
from math import sqrt, exp, pi

# Load a CSV file and return the dataset as a list of lists
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert a specific column in the dataset from string to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert a specific column in the dataset from string to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Split the dataset by class values and return a dictionary
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if class_value not in separated:
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated

# Calculate the mean of a list of numbers
def mean(numbers):
    return sum(numbers) / float(len(numbers))

# Calculate the standard deviation of a list of numbers
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x - avg) ** 2 for x in numbers]) / float(len(numbers) - 1)
    return sqrt(variance)

# Calculate the mean, standard deviation, and count for each column in the dataset
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del summaries[-1]
    return summaries

# Split dataset by class and calculate statistics for each row
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries

# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2] / float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

# Predict the class for a given row
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

# Naive Bayes Algorithm
def naive_bayes(train, test):
    summarize = summarize_by_class(train)
    predictions = list()
    for row in test:
        output = predict(summarize, row)
        predictions.append(output)
    return predictions

# Set the random seed
seed(1)

# Specify the filename of the CSV dataset
filename = '/content/winequality-red_final_1240.csv.csv'

# Load the CSV dataset
dataset = load_csv(filename)

# Convert the columns of the dataset from string to float
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)

# Convert the class column to integers
str_column_to_int(dataset, len(dataset[0]) - 1)

# Evaluate the Naive Bayes algorithm
n_folds = 5
scores = evaluate_algorithm(dataset, naive_bayes, n_folds)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))

Scores: [45.141065830721004, 48.275862068965516, 42.63322884012539, 43.260188087774296, 45.141065830721004]
Mean Accuracy: 44.890%


### Naive Bayes Prediction on the Wine quality red dataset


In [18]:
# Make Predictions with Naive Bayes On The Wine quality red dataset
from csv import reader
from math import sqrt
from math import exp
from math import pi

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
        print('[%s] => %d' % (value, i))  # Print mapping of class labels to integers
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split the dataset by class values, returns a dictionary
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if class_value not in separated:
            separated[class_value] = list()
        separated[class_value].append(vector)
    return separated

# Calculate the mean of a list of numbers
def mean(numbers):
    return sum(numbers) / float(len(numbers))

# Calculate the standard deviation of a list of numbers
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x - avg) ** 2 for x in numbers]) / float(len(numbers) - 1)
    return sqrt(variance)

# Calculate the mean, stdev and count for each column in a dataset
def summarize_dataset(dataset):
    summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
    del (summaries[-1])
    return summaries

# Split dataset by class then calculate statistics for each row
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries

# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

# Predict the class for a given row
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

# Calculate class probabilities for a given row
def calculate_class_probabilities(summaries, row):
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = 1
        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            x = row[i]
            probabilities[class_value] *= calculate_probability(x, mean, stdev)
    return probabilities

# Make a prediction with Naive Bayes on the Banknote Dataset
filename = '/content/winequality-red_final_1240.csv.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)
# Convert class column to integers
str_column_to_int(dataset, len(dataset[0]) - 1)
# Fit model
model = summarize_by_class(dataset)

# Define new records
rows = [
    [6.2, 3.4, 5.4, 2.3],
    [4.8, 3.0, 1.4, 0.2],
    [5.5, 2.6, 4.7, 1.5]
]

# Predict the labels
for row in rows:
    label = predict(model, row)
    print('Data=%s, Predicted: %s' % (row, label))

[5] => 0
[8] => 1
[4] => 2
[7] => 3
[6] => 4
[3] => 5
Data=[6.2, 3.4, 5.4, 2.3], Predicted: 5
Data=[4.8, 3.0, 1.4, 0.2], Predicted: 5
Data=[5.5, 2.6, 4.7, 1.5], Predicted: 5
